In [1]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Input
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy as acc
from keras.applications.vgg16 import VGG16
from flip_gradient import flip_gradient
from keras.backend import learning_phase
#from ourUtils import 

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


### lable modle without DA

In [2]:
def lable_model(l2_reg = 0.01, do_rate = 0, vgg_train = True):
    # Load the convolutional part of the VGG16 network 
    vgg16Conv = VGG16(weights='imagenet', include_top=False)

    # Input to network
    vggInput = Input(shape=(224, 224, 3), name='image_input')
    # Output of convolutional part
    output_vggConv = vggConv(vggInput)
    # Stack lable layers
    preDns = Flatten(name='preLp')(output_vggConv)
    dns1 = Dense(2048, activation='relu', kernel_initializer='glorot_normal', 
                 bias_initializer='glorot_normal', kernel_regularizer=l2(l=l2_reg), name='lpl1')(preDns)
    dns1Do = Dropout(rate=do_rate, seed=42, name='lpl1Do')(dns1)
    dns2 = Dense(1024, activation='relu', kernel_initializer='glorot_normal', 
                 bias_initializer='glorot_normal', kernel_regularizer=l2(l=l2_reg), name='lpl2')(dns1Do)
    modelOut = Dense(5, activation='softmax', kernel_initializer='glorot_normal', name='lplOut')(dns2)

    # Optimizer
    optimize = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    # Compile the model
    vggConvSleep.compile(loss='categorical_crossentropy', optimizer=optimize, metrics=['categorical_accuracy'])

    # Get model summary
    vggConvSleep.summary()
    
    return vggConvSleep

In [3]:
def DA_model(batch_size, l2_reg = 0.01, do_rate = 0, vgg_train = True):
    # Load the convolutional part of the VGG16 network 
    vgg16Conv = VGG16(weights='imagenet', include_top=False)

    # Input to network
    vggInput = Input(batch_shape=(batch_size,224,224,3), shape=(224, 224, 3), name='image_input')
    # Output of convolutional part
    output_vggConv = vgg16Conv(vggInput)
    # pre Dence layer
    preDns = Flatten(name='preDa')(output_vggConv)
    # Stack lable layers
    lplSlice = Lambda(lambda x: x[:int(batch_size/2), :] if learning_phase() else x , name='lplSplit')(preDns)
    lpl1 = Dense(2048, activation='relu', kernel_initializer='glorot_normal', 
                 bias_initializer='glorot_normal', kernel_regularizer=l2(l=l2_reg), name='lpl1')(lplSlice)
    lpl1Do = Dropout(rate=do_rate, seed=42, name='lpl1Do')(lpl1)
    lpl2 = Dense(1024, activation='relu', kernel_initializer='glorot_normal', 
                 bias_initializer='glorot_normal', kernel_regularizer=l2(l=l2_reg), name='lpl2')(lpl1Do)
    lplOut = Dense(5, activation='softmax', kernel_initializer='glorot_normal', name='lplOut')(lpl2)
    # Stack domain layers
    flipGrad = Lambda(lambda x: flip_gradient(x,1),name='flipGrad')(preDns)
    dpl1 = Dense(2048, activation='relu', kernel_initializer='glorot_normal', 
                 bias_initializer='glorot_normal', kernel_regularizer=l2(l=l2_reg), name='dpl1')(flipGrad)
    dpl1Do = Dropout(rate=do_rate, seed=42, name='dpl1Do')(dpl1)
    dpl2 = Dense(1024, activation='relu', kernel_initializer='glorot_normal', 
                 bias_initializer='glorot_normal', kernel_regularizer=l2(l=l2_reg), name='dpl2')(dpl1Do)
    dplOut = Dense(5, activation='softmax', kernel_initializer='glorot_normal', name='dplOut')(dpl2)
    
    #stitch modle together
    vggConvSleep = Model(inputs=vggInput, outputs=[lplOut, dplOut])
    
    # Optimizer
    optimize = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    # Compile the model
    vggConvSleep.compile(loss={'lplOut':'categorical_crossentropy','dplOut':'categorical_crossentropy'},
                         optimizer=optimize, metrics=['categorical_accuracy'])

    # Get model summary
    vggConvSleep.summary()
    
    return vggConvSleep

In [4]:
modle = DA_model(40)

TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

In [ ]:
lable_model()
    